In [19]:
import pandas as pd
import numpy as np

#load data
train_data = pd.read_csv('./train.csv')
test_data = pd.read_csv('./test.csv')

train_data.head()

,AMB_TEMP,CO,NO,NO2,NOx,O3,PM10,WS_HR,RAINFALL,RH,SO2,WD_HR,WIND_DIREC,WIND_SPEED,PM2.5
0,10.8,0.32,1.7,8.6,10.3,22.9,21,0.6,0.0,71,1.9,172,171,0.6,15
1,10.8,0.27,1.6,6.2,7.8,23.8,20,1.4,0.0,71,1.7,161,129,1.8,13
2,11.0,0.25,0.9,5.4,6.3,27.4,21,0.8,0.0,68,1.6,152,147,1.5,12
3,11.0,0.23,0.7,3.1,3.8,29.5,21,1.8,0.0,68,1.6,138,145,1.7,9
4,11.3,0.22,0.8,2.9,3.8,30.7,16,1.9,0.0,67,1.6,140,139,1.7,9


In [20]:
#資料清理
train_data_filtered = train_data[train_data['PM2.5'] <= 100]

#相關性分析
correlation_with_pm25 = train_data_filtered.corr()['PM2.5']
print(correlation_with_pm25)

KeyError: "None of [Index([0.17614746492930122,  0.6591476679747462, 0.22721914669686602,\n         0.554273687399106,  0.5136501396100956, 0.23392394378559644,\n         0.818868213770211, 0.10204740464346891, 0.06080122144870972,\n        0.0815764285190882, 0.36133341636422167, 0.17193239718636125,\n       0.13765835101624818, 0.10119696040759044,                 1.0],\n      dtype='float64')] are in the [index]"

In [3]:
#選擇相關性大於0.5的features
selected_features = correlation_with_pm25[correlation_with_pm25.abs()>0.5].index.tolist()
print(selected_features)

['CO', 'NO2', 'NOx', 'PM10', 'PM2.5']


In [4]:
#將選擇的feature的值提取出來
X_selected = train_data_filtered[selected_features].values

#preparing training data
train_X = []
train_y = []
for i in range(len(X_selected) -8):
    train_X.append(X_selected[i:i+8].flatten())
    train_y.append(X_selected[i+8, -1])

train_X = np.array(train_X)
train_y = np.array(train_y)

In [5]:
#Normalization
mean_train = np.mean(train_X, axis=0)
std_train = np.std(train_X, axis=0)
train_X_normalized = (train_X - mean_train) / (std_train + 1e-8)

In [6]:
#Adam 超參數
alpha = 0.001
beta1 = 0.9
beta2 = 0.999
epsilon = 1e-8
m = 0
v = 0
t = 0

In [15]:
#初始化Weights, bias
np.random.seed(0)
W = np.zeros(train_X_normalized.shape[1])
print(train_X_normalized.shape)
print(W.shape)
b = np.random.randn()

num_epochs = 1000

(5766, 40)
(40,)


In [17]:
count = 0
for  i in range(0, 720-7, 8):
    print(i)
    count+=1
print(count)    

0
8
16
24
32
40
48
56
64
72
80
88
96
104
112
120
128
136
144
152
160
168
176
184
192
200
208
216
224
232
240
248
256
264
272
280
288
296
304
312
320
328
336
344
352
360
368
376
384
392
400
408
416
424
432
440
448
456
464
472
480
488
496
504
512
520
528
536
544
552
560
568
576
584
592
600
608
616
624
632
640
648
656
664
672
680
688
696
704
712
90


In [13]:
#Training
for epoch in range(num_epochs):
    y_pred = np.dot(train_X_normalized, W) + b
    error = y_pred - train_y
    gradient_W = (2/len(train_X_normalized)) * np.dot(train_X_normalized.T, error)
    gradient_b = (2/len(train_X_normalized)) * error.sum()
    
    t += 1
    m = beta1 * m + (1 - beta1) * gradient_W
    v = beta2 * v + (1 - beta2) * (gradient_W ** 2)
    m_corrected = m / (1 - beta1 ** t)
    v_corrected = v / (1 - beta2 ** t)
    W -= alpha * m_corrected / (np.sqrt(v_corrected) + epsilon)

    m_b = beta1 * m + (1 - beta1) * gradient_b
    v_b = beta2 * v + (1 - beta2) * (gradient_b ** 2)
    m_b_corrected = m_b / (1 - beta1 ** t)
    v_b_corrected = v_b / (1 - beta2 ** t)
    b -= alpha * m_b_corrected / (np.sqrt(v_b_corrected) + epsilon)



ValueError: operands could not be broadcast together with shapes (5766,) (40,) 

In [ ]:
#predict
